In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)
trainset,validationset = random_split(trainset,[round(0.9 * len(trainset)), round(0.1 * len(trainset))])

Files already downloaded and verified
Files already downloaded and verified


In [ ]:



trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

In [ ]:




class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3072, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 10)
        
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
def count_model_params(model):
    """ Counting the number of learnable parameters in a nn.Module """
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_params


In [ ]:
model = Classifier()
# model = model.to(device)

# criterion = nn.NLLLoss().to(device)
criterion = nn.NLLLoss()
lr = 0.00083
optimizer = optim.Adam(model.parameters(), lr= lr)

epochs = 50
steps = 0
cm = [ [ 0 for i in range(10) ] for j in range(10) ]


train_losses, test_losses = [], []
train_acc, test_acc =[],[]
  
for e in range(epochs):

    running_loss = 0
    accuracy2 = 0

    for images, labels in trainloader:
        # images= images.to(device)
        # labels = labels.to(device)
        
        optimizer.zero_grad()
        log_ps = model(images)        
        loss = criterion(log_ps, labels)
        loss.backward()
        
        optimizer.step()
        with torch.no_grad(): 
          model.eval()
          log_ps2 = log_ps.clone().detach()               
          ps2 = torch.exp(log_ps2)
          top_p2, top_class2 = ps2.topk(1, dim=1)
          equals2 = top_class2 == labels.view(*top_class2.shape)
          accuracy2 += torch.mean(equals2.type(torch.FloatTensor))



        model.train()        
        running_loss += loss.item()

    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            
            model.eval()
            for images, labels in testloader:
                # images= images.to(device)
                # labels = labels.to(device)             
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                # cm += confusion_matrix(labels.view(*top_class.shape).cpu(), top_class.cpu())
                

        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        train_acc.append(accuracy2/len(trainloader))
        test_acc.append(accuracy/len(testloader))
        # Norm_grad.append(total_norm)
        

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Train Accuracy: {:.3f}".format(accuracy2/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
# print(cm)
        


Epoch: 1/50..  Training Loss: 1.921..  Train Accuracy: 0.294 Test Loss: 1.742..  Test Accuracy: 0.373
Epoch: 2/50..  Training Loss: 1.722..  Train Accuracy: 0.378 Test Loss: 1.661..  Test Accuracy: 0.405
Epoch: 3/50..  Training Loss: 1.632..  Train Accuracy: 0.414 Test Loss: 1.609..  Test Accuracy: 0.422
Epoch: 4/50..  Training Loss: 1.571..  Train Accuracy: 0.434 Test Loss: 1.562..  Test Accuracy: 0.435
Epoch: 5/50..  Training Loss: 1.530..  Train Accuracy: 0.454 Test Loss: 1.506..  Test Accuracy: 0.467
Epoch: 6/50..  Training Loss: 1.489..  Train Accuracy: 0.468 Test Loss: 1.510..  Test Accuracy: 0.465
Epoch: 7/50..  Training Loss: 1.453..  Train Accuracy: 0.481 Test Loss: 1.476..  Test Accuracy: 0.469
Epoch: 8/50..  Training Loss: 1.429..  Train Accuracy: 0.489 Test Loss: 1.453..  Test Accuracy: 0.490
Epoch: 9/50..  Training Loss: 1.397..  Train Accuracy: 0.501 Test Loss: 1.434..  Test Accuracy: 0.484
Epoch: 10/50..  Training Loss: 1.374..  Train Accuracy: 0.512 Test Loss: 1.441..  